In [ ]:
## Notebook to run live estimation of image information saturation followed by QnR optimization on imswitch virtual micrsocope
# 2024-07-19
# Author: Hannah Heil



In [ ]:
!pip install -q nanopyx

In [ ]:
# Imports

import nanopyx
from nanopyx.core.transform._le_esrrf import eSRRF
from nanopyx import eSRRF as run_eSRRF
from nanopyx.core.utils.timeit import timeit2
from nanopyx import run_esrrf_parameter_sweep
#from nanopyx.core.transform.sr_temporal_correlations import calculate_eSRRF_temporal_correlations
import numpy as np
from tqdm import tqdm
import imswitch
from imswitch.imcontrol.model.managers.rs232.VirtualMicroscopeManager import VirtualMicroscopy
import matplotlib.pyplot as plt
#for SSIM calculation
from skimage.metrics import structural_similarity as ssim
from IPython.display import display, clear_output
from PIL import Image


In [ ]:
## Acquire an image img_stack from the imswitch virtual microscope until sampling is sufficient 

# Images are acquired with the "eSRRF" mode of the virtual microscope with will output the eSRRF reconstruction of all images 
# that have been acquired so far as well as the raw image img_stack. 
# The mean square error (MSE) is calculated betwen consecutive eSRRF reconstructions to see the trend of the image information. 
# If the difference between the most recent frames (mse_new) does not change more than 10% of the differenct of thoses preceding these (mse_old) 
# the image information saturation is reached and the loop ends. 
# If the MSE in two consecutive frames is smaller than 10% of the MSE in the first frame, the image information saturation is reached and the loop ends.
#___________________________________________#
##Functions: 

#Calculate MSE
@timeit2
def calculate_mse(imageA, imageB):
    # the 'Mean Squared Error' between the two images is the
    # sum of the squared difference between the two images;
    # NOTE: the two images must have the same dimension
    # Check if the images have the same dimensions
    if imageA.shape != imageB.shape:
        raise ValueError("Input images must have the same dimensions.")

    err_value = np.sum((imageA.astype("float") - imageB.astype("float")) ** 2)
    err_value /= float(imageA.shape[0] * imageA.shape[1])
    # return the MSE, the lower the error, the more "similar"
    # the two images are
    return err_value

#Calculate SSIM

@timeit2
def calculate_ssim(imageA, imageB):
    # Check if the images have the same dimensions
    if imageA.shape != imageB.shape:
        raise ValueError("Input images must have the same dimensions.")
    # Get data range
    range=np.max(imageA)-np.min(imageA)
    # Compute the SSIM between the two images
    ssim_value, _ = ssim(imageA, imageB, full=True, data_range=range)

    return ssim_value

# Function to update the plot
@timeit2
def update_plot(x_data, y_data):
    line.set_xdata(x_data)
    line.set_ydata(y_data)
    ax.relim()
    ax.autoscale_view()
    fig.canvas.draw()
    fig.canvas.flush_events()
    clear_output(wait=True)
    display(fig)

#___________________________________________#

## Main:
# Initialize the plot
plt.ion()
fig, ax = plt.subplots()
line, = ax.plot([], [])

# Initialize the virtual microscope
microscope = VirtualMicroscopy(filePath="smlm") # for now options are: example or SMLM
microscope.camera.return_raw = True
microscope.illuminator.set_intensity(intensity=5000)
microscope.positioner.move(x=1400, y=-800, z=0, is_absolute=True)
#microscope.camera.crop(hpos=0, vpos=0, hsize=1000, vsize=1000) # Optional cropping of the camera ROI, not implemented yet for virtual camera


img_stack = np.expand_dims(microscope.camera.getLast(), axis=0) #get first frame
rgc_stack = np.expand_dims(run_eSRRF(img_stack[-1])[0], axis=0)

img_stack = np.concatenate((img_stack, np.expand_dims(microscope.camera.getLast(), axis=0)), axis=0)
rgc_stack = np.concatenate((rgc_stack, np.expand_dims(run_eSRRF(img_stack[-1], _force_run_type="threaded")[0], axis=0)), axis=0)

mse_new=calculate_mse(np.mean(rgc_stack[:-1], axis=0), np.mean(rgc_stack, axis=0))
#ssim_new=calculate_ssim(eSRRF_AVG_old, eSRRF_AVG_new) # otional SSIM calculation, more robust but much slower
mse_array=[]
#ssim_array=[] # for SSIM option
mse_array=[mse_new]
#ssim_array=[ssim_new] # fro SSIM option
x_data=[1] #counter for the frame number

count=1
# run acquisition loop until saturation threshold is reached
while True:
    count=count+1
    mse_old=mse_new
    #ssim_old=ssim_new #for SSIM option
    img_stack = np.concatenate((img_stack, np.expand_dims(microscope.camera.getLast(), axis=0)), axis=0)
    tmp = np.expand_dims(run_eSRRF(img_stack[-1], _force_run_type="threaded")[0], axis=0)
    rgc_stack = np.concatenate((rgc_stack, tmp), axis=0)
    if count % 10 == 0: #update plot every 10 frames
        x_data.append(count)
        mse_new=calculate_mse(np.mean(rgc_stack[:-1], axis=0), np.mean(rgc_stack, axis=0))
        #ssim_new=calculate_ssim(eSRRF_AVG_old, eSRRF_AVG_new) # for SSIM option
        mse_array.append(mse_new)
        #ssim_array.append(ssim_new)   # for SSIM option
        update_plot(x_data, mse_array) 
        if abs(mse_old-mse_new) < 0.1*mse_old:
            print("treshold:", 0.1*mse_old)
            print("reached treshold at:", mse_old-mse_new)
            break

plt.ioff()
        

In [ ]:
## eSRRF Parameter optimization
# parameter sweep runs on central 100x100 pixel ROI of the image img_stack acquired in the previous celland outputs the QnR map for each radius and sensitivity.
# The resoluton estimation can be based on FRC or decorrelation (recomended).

##Parameters : 
radii = range(3,8,1) # radius sweep range (start, stop, step)
sensitivities = range(5,11,1) # sensitivity sweep range (start, stop, step)
global_mag = 2 #magnification   
temp_corr = "AVG" # temporal reconstruction mode: "AVG", "VAR" or "TAC2
use_decorr = True # resolution estimation based on FRC (False) or decorrelation (True)

global g_temp_corr
if temp_corr == "AVG":
    g_temp_corr = 1
elif temp_corr == "VAR":
    g_temp_corr = 2
elif temp_corr == "TAC2":
    g_temp_corr = 3

print(img_stack.shape)
img_stack=img_stack.reshape(microscope.camera.frameNumber,microscope.camera.SensorHeight,microscope.camera.SensorWidth)
center_x=int(abs(img_stack.shape[1]/2))
center_y=int(abs(img_stack.shape[2]/2))
print(center_x, center_y)
dataset_original = np.ascontiguousarray(img_stack[:,center_x-50:center_x+50,center_y-50:center_y+50], dtype=np.float32) #restrition to the center 100x100 pixel ROI
#dataset_original=img_stack
print(dataset_original.shape)
param_sweep_out = run_esrrf_parameter_sweep(dataset_original, magnification=global_mag, sensitivities=sensitivities, radii=radii, temporal_correlation=temp_corr, plot_sweep=True, return_qnr=True, use_decorr=use_decorr)

sens_idx, rad_idx = np.unravel_index(np.argmax(param_sweep_out), param_sweep_out.shape)
global optimal_sensitivity
optimal_sensitivity = sensitivities[sens_idx]
global optimal_radius
optimal_radius = radii[rad_idx]
print(f"Optimal sensitivity is: {optimal_sensitivity}; and optimal radius is: {optimal_radius}")


In [ ]:
# Test visualization of the last two eSRRF frames
print(rgc_stack[-2].shape)
print(rgc_stack[-1].shape)
plt.imshow(rgc_stack[-2], cmap='gray',vmin=None, vmax=None)
plt.show()
plt.imshow(rgc_stack[-1], cmap='gray',vmin=None, vmax=None)
plt.show()